In [1]:
import os
if "models" not in os.listdir("."):
    os.chdir("../..")

In [7]:
import json

with open("cleanup_results_final.jsonl") as f:
    lines = f.readlines()
    results = [json.loads(line) for line in lines]

In [32]:
# task_name = "antonyms"

# results = [r for r in results if r["task"] == task_name]


import pandas as pd

df = pd.DataFrame(results)

melted_df = df.melt(id_vars=["layer", "task"], value_vars=["loss", "tv_loss", "ito_loss", "recon_loss"], var_name="loss_type", value_name="loss value")

In [23]:
melted_df.head()

,layer,task,loss_type,loss value
0,1,location_continent,loss,10.3125
1,2,location_continent,loss,10.2500
2,3,location_continent,loss,10.1875
3,4,location_continent,loss,10.3750
4,5,location_continent,loss,10.1875


In [24]:
import plotly.express as px

task_name = "algo_last"

px.line(melted_df[melted_df["task"] == task_name], x="layer", y="loss value", color="loss_type")

In [36]:
task_names = df["task"].unique()
layers = sorted(df["layer"].unique())

task_losses = {task_name: {loss_type: df[df["task"] == task_name][loss_type].to_numpy() for loss_type in ["loss", "tv_loss"]} for task_name in task_names}

normalized_losses = {task_name: {loss_type: (task_losses[task_name][loss_type] - task_losses[task_name][loss_type].min()) / (task_losses[task_name][loss_type].max() - task_losses[task_name][loss_type].min()) for loss_type in ["loss", "tv_loss"]} for task_name in task_names}

In [43]:
heatmap = pd.DataFrame({task_name: normalized_losses[task_name]["loss"] for task_name in task_names}, index=layers)
px.imshow(heatmap)

In [44]:
%load_ext autoreload
%autoreload 2
import penzai
import jax_smi
jax_smi.initialise_tracking()
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [45]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2b-it.gguf", from_type="gemma", load_eager=True, device_map="tpu:0")

In [46]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

In [47]:
from sprint.task_vector_utils import load_tasks, ICLRunner
tasks = load_tasks()

fatal: destination path 'data/itv' already exists and is not an empty directory.


In [48]:
# task_names = ["en_es", "antonyms", "person_profession", "es_en", "present_simple_gerund", "present_simple_past_simple", "person_profession", "person_language", "country_capital", "football_player_position"]
# task_name = task_names[1]
task_names = list(tasks.keys())

In [49]:
import jax.numpy as jnp
import jax


from sprint.task_vector_utils import ICLRunner, logprob_loss, get_tv, make_act_adder
from micrlhf.llama import LlamaBlock
from micrlhf.sampling import sample, jit_wrapper


get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_pre_{i}"),
        x
    ])
)
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
get_resids_call = jit_wrapper.Jitted(get_resids)



def tokenized_to_inputs(input_ids, attention_mask):
    token_array = jnp.asarray(input_ids)
    token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq").untag("batch").tag("batch")

    mask_array = jnp.asarray(attention_mask, dtype=jnp.bool)
    mask_array = jax.device_put(mask_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    mask_array = pz.nx.wrap(mask_array, "batch", "seq").untag("batch").tag("batch")

    inputs = llama.inputs.from_basic_segments(token_array)
    return inputs


In [52]:
from sprint.icl_sfc_utils import AblatedModule
layer = 12
mask_name = "arrow"

In [54]:
from micrlhf.utils.load_sae import get_nev_it_sae_suite


sae = get_nev_it_sae_suite(layer=layer)

In [58]:
import numpy as np

features = []

for task_name in task_names:
    task_results = [result for result in results if result["task"] == task_name and result["layer"] == layer]

    for result in task_results:
        w = np.array(result["weights"])
        # s = jax.nn.softplus(sae["s_gate"]) * sae["scaling_factor"]
        # threshold = jnp.maximum(0, sae["b_gate"] - sae["b_enc"] * s)
        w = w * (w > 0)

        # print(threshold)

        features += np.nonzero(w)[0].tolist()

features = list(set(features))

len(features)

129

In [60]:
import dataclasses
from tqdm.auto import tqdm
from functools import partial
from micrlhf.utils.activation_manipulation import add_vector

task_losses_positive = {}

n_few_shots, batch_size, max_seq_len = 20, 16, 256
seed = 10

prompt = "Follow the pattern:\n{}"

def make_taker(llama, layer):
    taker = jit_wrapper.Jitted(llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(
        lambda i, x: x if i >= layer else pz.nn.Identity()
    ).select().at_instances_of(pz.nn.EmbeddingLookup).apply(lambda _: pz.nn.Identity())
                    .select().at_instances_of(pz.nn.ConstantRescale).pick_nth_selected(0).apply(lambda _: pz.nn.Identity()))

    return taker

taker = make_taker(llama, layer)

for task_name in tqdm(task_names):

    sep = 3978
    pad = 0


    pairs = list(tasks[task_name].items())

    n_shot = n_few_shots - 1
    if task_name.startswith("algo"):
        n_shot = 8

    runner = ICLRunner(task_name, pairs, batch_size=batch_size, n_shot=1, max_seq_len=max_seq_len, seed=seed, prompt=prompt)

    tokenized = runner.get_tokens([
        x[:n_few_shots] for x in runner.train_pairs
    ], tokenizer)

    inputs = tokenized_to_inputs(**tokenized)
    train_tokens = tokenized["input_ids"]

    _, all_resids = get_resids_call(inputs)

    scale = 25

    resids = all_resids[layer].value.unwrap("batch", "seq", "embedding")

    mask = train_tokens == sep
    col_indices = jnp.arange(mask.shape[1])
    col_indices_broadcasted = mask * col_indices
    sorted_indices = jnp.sort(col_indices_broadcasted, axis=1, descending=True)
    positions = sorted_indices[:, :1]
    
    def steer_with_direction(direction):
        direction = direction / jnp.linalg.norm(direction)
        direction = direction * scale
        
        modified = jax.vmap(lambda a, b: a.at[b].add(direction))(
            resids, positions
        )
        modified = pz.nx.wrap(modified, "batch", "seq", "embedding")

        _inputs = dataclasses.replace(inputs, tokens=modified)
        logits = taker(_inputs).unwrap("batch", "seq", "vocabulary")

        return logprob_loss(logits, train_tokens, sep=sep, pad_token=pad, n_first=2)

    task_losses_positive[task_name] = [[steer_with_direction(sae["W_dec"][feature]).tolist() for feature in tqdm(features)]]

    logits = llama(inputs)

    logits = logits.unwrap("batch", "seq", "vocabulary")

    task_losses_positive[task_name].append(logprob_loss(logits, train_tokens, sep=sep, pad_token=pad, n_first=2).tolist())

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/129 [00:00<?, ?it/s]

  0%|          | 0/129 [00:00<?, ?it/s]

  0%|          | 0/129 [00:00<?, ?it/s]

  0%|          | 0/129 [00:00<?, ?it/s]

...

  0%|          | 0/129 [00:00<?, ?it/s]

  0%|          | 0/129 [00:00<?, ?it/s]

  0%|          | 0/129 [00:00<?, ?it/s]

  0%|          | 0/129 [00:00<?, ?it/s]

  0%|          | 0/129 [00:00<?, ?it/s]

In [61]:
task_losses_positive["antonyms"]

[[6.03125,
  6.0,
  5.03125,
  4.625,
  6.21875,
  4.71875,
  3.53125,
  6.3125,
  4.78125,
  9.5,
  6.15625,
  3.890625,
  11.0,
  7.03125,
  5.59375,
  5.0625,
  4.75,
  6.6875,
  5.6875,
  5.1875,
  5.5625,
  11.5,
  4.34375,
  6.15625,
  3.625,
  8.375,
  4.1875,
  14.1875,
  10.0,
  10.0,
  9.0,
  4.75,
  13.1875,
  3.1875,
  2.59375,
  4.375,
  5.9375,
  8.0625,
  6.1875,
  3.671875,
  5.75,
  4.9375,
  17.625,
  4.84375,
  4.78125,
  5.25,
  9.6875,
  7.28125,
  7.15625,
  9.1875,
  5.125,
  15.25,
  9.0,
  7.25,
  6.125,
  16.0,
  5.65625,
  9.8125,
  5.78125,
  15.3125,
  5.8125,
  3.984375,
  8.8125,
  8.4375,
  6.8125,
  5.1875,
  10.0,
  9.625,
  7.9375,
  4.09375,
  4.65625,
  7.28125,
  12.3125,
  5.34375,
  13.4375,
  4.25,
  4.78125,
  7.53125,
  1.40625,
  11.0625,
  6.78125,
  7.75,
  6.34375,
  18.875,
  3.71875,
  9.75,
  16.5,
  6.5,
  8.8125,
  4.5,
  4.25,
  6.4375,
  10.875,
  7.90625,
  3.203125,
  9.875,
  6.34375,
  6.75,
  4.0,
  7.5,
  4.78125,
  5.0,
  14.0,
  13.25,
  10.8125,
  5.96875,
  8.8125,
  4.78125,
  6.75,
  10.1875,
  7.625,
  10.5625,
  8.75,
  7.0,
  15.125,
  9.6875,
  6.03125,
  5.40625,
  9.5625,
  6.15625,
  3.53125,
  3.625,
  5.6875,
  6.375,
  8.5625,
  3.265625,
  7.53125,
  7.71875,
  8.0],
 4.53125]

In [119]:
import plotly.express as px
import pandas as pd
import numpy as np

normalized_losses = {}

drop_features = []
drop_ids = [features.index(feature) for feature in drop_features]

features_dropped = [feature for feature in features if feature not in drop_features]

for task_name, losses in task_losses_positive.items():
    base_loss = losses[1]
    losses = losses[0]
    losses = np.array(losses)
    losses = np.delete(losses, drop_ids)

    losses = np.minimum(losses, base_loss)

    losses = (losses - base_loss) / base_loss
    
    max_loss = np.max(losses)
    min_loss = np.min(losses)

    losses = (losses - min_loss) / (max_loss - min_loss)
    
    # mean_loss = np.mean(losses - base_loss)

    # base_acc = losses[1][1]

    # accs = [base_acc - loss[1] for loss in losses[0]]
    normalized_losses[task_name] = losses


heatmap = np.zeros((len(task_names), len(features_dropped)))

for i, task_name in enumerate(task_names):
    for j, feature in enumerate(features_dropped):
        heatmap[i, j] = normalized_losses[task_name][j]

# heatmap /= np.mean(heatmap, axis=0, keepdims=True)



# heatmap = np.where(heatmap > 0, np.log(heatmap), -10)
# heatmap[np.isnan(heatmap)] = np.min(heatmap[np.isfinite(heatmap)])
# heatmap[np.isinf(heatmap)] = np.max(heatmap[np.isfinite(heatmap)])

# heatmap = np.clip(heatmap, -5, 5)


fig = px.imshow(heatmap, x=[str(x) for x in features_dropped], y=task_names)

fig.show()


In [120]:
features_dropped.index(7491)

69

In [121]:
mean_heatmap = np.mean(heatmap, axis=0)

drop_features = np.where(mean_heatmap < 0.9)[0]

drop_features = [features_dropped[x] for x in drop_features]

drop_features

[26127,
 6171,
 10787,
 30247,
 3113,
 9782,
 32320,
 15947,
 23682,
 12943,
 16016,
 13458,
 30888,
 19112,
 11491,
 23312,
 7967,
 7491,
 1878,
 20832,
 6503,
 26987,
 2939,
 31620,
 11172,
 25534,
 6594,
 5579,
 23514,
 6629,
 18416]

In [115]:
heatmap.shape

(23, 129)

In [116]:
(22 + 0.1)/23

0.9608695652173914

In [127]:
import plotly.express as px
import pandas as pd
import numpy as np

normalized_losses = {}
drop_ids = [features.index(feature) for feature in drop_features]

features_dropped = [feature for feature in features if feature not in drop_features]

for task_name, losses in task_losses_positive.items():
    base_loss = losses[1]
    losses = losses[0]
    losses = np.array(losses)
    losses = np.delete(losses, drop_ids)


    losses = np.minimum(losses, base_loss * 1.5)

    losses = (losses - base_loss) / base_loss
    
    max_loss = np.max(losses)
    min_loss = np.min(losses)

    losses = (losses - min_loss) / ((max_loss - min_loss) + 1e-6)
    
    # mean_loss = np.mean(losses - base_loss)

    # base_acc = losses[1][1]

    # accs = [base_acc - loss[1] for loss in losses[0]]
    normalized_losses[task_name] = losses


heatmap = np.zeros((len(task_names), len(features_dropped)))

for i, task_name in enumerate(task_names):
    for j, feature in enumerate(features_dropped):
        heatmap[i, j] = normalized_losses[task_name][j]

# heatmap /= np.mean(heatmap, axis=0, keepdims=True)



# heatmap = np.where(heatmap > 0, np.log(heatmap), -10)
# heatmap[np.isnan(heatmap)] = np.min(heatmap[np.isfinite(heatmap)])
# heatmap[np.isinf(heatmap)] = np.max(heatmap[np.isfinite(heatmap)])

# heatmap = np.clip(heatmap, -5, 5)

heatmap = np.log(heatmap + 1)


fig = px.imshow(heatmap, x=[str(x) for x in features_dropped], y=task_names, color_continuous_scale='YlGnBu')

fig.show()

In [125]:
task_losses_positive["present_simple_past_perfect"]

[[10.3125,
  4.40625,
  4.75,
  6.6875,
  6.0625,
  5.15625,
  5.96875,
  5.9375,
  5.15625,
  7.8125,
  6.84375,
  4.75,
  7.125,
  7.625,
  6.25,
  4.875,
  6.5,
  5.1875,
  4.25,
  4.875,
  5.75,
  9.625,
  6.03125,
  10.5,
  6.6875,
  10.625,
  5.46875,
  9.1875,
  9.625,
  11.125,
  13.125,
  6.90625,
  4.21875,
  5.71875,
  4.75,
  4.71875,
  6.0625,
  5.625,
  5.03125,
  6.6875,
  4.96875,
  6.65625,
  11.1875,
  9.8125,
  7.375,
  4.875,
  10.125,
  6.75,
  7.96875,
  6.8125,
  5.625,
  14.75,
  13.1875,
  7.9375,
  5.28125,
  13.3125,
  6.3125,
  6.71875,
  5.1875,
  19.375,
  6.5,
  6.65625,
  5.25,
  4.6875,
  6.03125,
  7.9375,
  11.625,
  10.4375,
  6.5,
  6.09375,
  6.9375,
  10.0,
  17.0,
  5.15625,
  13.625,
  6.59375,
  5.1875,
  7.1875,
  5.34375,
  13.0,
  4.96875,
  7.3125,
  4.75,
  8.75,
  5.09375,
  5.0,
  17.875,
  6.09375,
  7.1875,
  5.34375,
  6.9375,
  8.5625,
  12.8125,
  5.0625,
  12.75,
  10.6875,
  9.0625,
  10.5,
  5.625,
  11.25,
  6.09375,
  7.1875,
  9.25,
  16.375,
  9.5625,
  5.78125,
  7.125,
  8.3125,
  10.0625,
  10.5625,
  5.125,
  3.53125,
  8.1875,
  6.28125,
  6.03125,
  11.875,
  6.59375,
  4.84375,
  8.875,
  6.1875,
  5.84375,
  5.03125,
  6.78125,
  6.25,
  4.25,
  8.8125,
  6.40625,
  8.375,
  4.0],
 3.84375]

In [84]:
mean_heatmap

array([0.10833333, 0.02105735, 0.0625    , 0.10611511, 0.0734767 ,
       0.07401033, 0.05617978, 0.05755396, 0.03819095, 0.03888889,
       0.19101124, 0.05395683, 0.22682446, 0.        , 0.06231156,
       0.0242411 , 0.10215054, 0.10453649, 0.04536489, 0.0094086 ,
       0.09346734, 0.13169643, 0.07777778, 0.22767857, 0.0915047 ,
       0.225     , 0.00416667, 0.00414938, 0.13403509, 0.2040201 ,
       0.27482014, 0.        , 0.        , 0.07482014, 0.06797853,
       0.05241935, 0.10663082, 0.01258993, 0.04496403, 0.02242991,
       0.04120603, 0.06919643, 0.39568345, 0.01032702, 0.02329749,
       0.08481262, 0.00280407, 0.09849246, 0.10555556, 0.04676259,
       0.01798561, 0.15654952, 0.26517572, 0.20143885, 0.07194245,
       0.617357  , 0.10267857, 0.        , 0.05527638, 0.15833333,
       0.13169643, 0.        , 0.01798561, 0.0729783 , 0.15779093,
       0.03718638, 0.03472222, 0.26111111, 0.13824561, 0.        ,
       0.10653266, 0.32070175, 0.        , 0.        , 0.2       ,
       0.02284946, 0.04856115, 0.09444444, 0.        , 0.4       ,
       0.08140704, 0.15107914, 0.03888889, 0.        , 0.08243728,
       0.00703518, 0.1522807 , 0.13632653, 0.04723618, 0.        ,
       0.04444444, 0.28801431, 0.375     , 0.06834532, 0.10286225,
       0.36628111, 0.12777778, 0.12096774, 0.01299283, 0.1       ,
       0.15770609, 0.16944444, 0.36094675, 0.        , 0.        ,
       0.03125   , 0.        , 0.17741935, 0.21111111, 0.11157895,
       0.        , 0.        , 0.13258786, 0.02817209, 0.05737705,
       0.47406082, 0.0031362 , 0.00277778, 0.00479233, 0.00138889,
       0.11079137, 0.01478495, 0.16853933, 0.02688172, 0.        ,
       0.07083333, 0.07222222, 0.05591054, 0.        ])

In [71]:
import plotly.express as px
import pandas as pd
import numpy as np

normalized_losses = {}

drop_features = []
drop_ids = [features.index(feature) for feature in drop_features]

features_dropped = [feature for feature in features if feature not in drop_features]

for task_name, losses in negative_task_losses.items():
    base_loss = losses[1]
    losses = [x[0].tolist() for x in losses[0]]
    losses = np.array(losses)
    losses = np.delete(losses, drop_ids)

    losses = losses - base_loss[0]
    
    max_loss = np.max(losses)
    min_loss = np.min(losses)

    losses = (losses - min_loss) / (max_loss - min_loss)
    
    # mean_loss = np.mean(losses - base_loss)

    # base_acc = losses[1][1]

    # accs = [base_acc - loss[1] for loss in losses[0]]
    normalized_losses[task_name] = losses


heatmap = np.zeros((len(task_names), len(features_dropped)))

for i, task_name in enumerate(task_names):
    for j, feature in enumerate(features_dropped):
        heatmap[i, j] = normalized_losses[task_name][j]

heatmap /= np.mean(heatmap, axis=0, keepdims=True)



# heatmap = np.where(heatmap > 0, np.log(heatmap), -10)
# heatmap[np.isnan(heatmap)] = np.min(heatmap[np.isfinite(heatmap)])
# heatmap[np.isinf(heatmap)] = np.max(heatmap[np.isfinite(heatmap)])

# heatmap = np.clip(heatmap, -5, 5)


fig = px.imshow(heatmap, x=[str(x) for x in features_dropped], y=task_names)

fig.show()


NameError: name 'negative_task_losses' is not defined